In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import warnings
import geopandas as gpd
from scipy.cluster import hierarchy
from sklearn import metrics
from scipy.cluster import vq
import sklearn.cluster as skc
import seaborn as sns
import pathlib
import pytest

import numpy as np
import pandas as pd
import geopandas as gpd 
import xarray as xr 
from shapely.geometry import Polygon, MultiPolygon
from collections import namedtuple

import FINE.spagat.dataset as spd
import FINE.spagat.representation as spr

import FINE as fn
import FINE.spagat.dataset as spd
import FINE.spagat.grouping as spg
import FINE.spagat.representation as spr 
import FINE.spagat.grouping_utils as gu
from FINE.IOManagement import dictIO
import FINE.IOManagement.xarrayIO as xrIO

from MATES import OptimizationManager



In [3]:
# Input files

INPUT_PATH = 'C:/Users/s.patil/Documents/code/fine/examples/Multi-regional_Energy_System_Workflow/InputData/EuropeanModelData'

input_data = os.path.join(INPUT_PATH,'new_EuropeanScenario_Ch5_Section1_DGC.nc')
shapefilePath = os.path.join(INPUT_PATH,'supregions.shp')



In [4]:
opM = OptimizationManager(input_data,
                          commodityUnitsDict={'electricity': 'GW_el', 
                                              'hydrogen': 'GW_h2', 
                                              'water': 'GW_wt', 
                                              'waterRes':'GW_wt', 
                                              'biomass':'GW_bio'})

In [5]:
esM_Europe = opM.setup()

The distances of a component are set to a normalized value of 1.


In [6]:
gdfRegions = gpd.read_file(shapefilePath)

In [7]:
sds = spd.SpagatDataset()
esm_dict, comp_dict = dictIO.exportToDict(esM_Europe)
sds.xr_dataset = xrIO.dimensional_data_to_xarray_dataset(esm_dict, comp_dict)

 c:\users\s.patil\documents\code\fine\FINE\utils.py:1049: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1049: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1049: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1049: UserWarning:Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
 c:\users\s.patil\documents\code\fine\FINE\utils.py:1049: UserWarning:Pandas doesn't allow columns to be created via a n

In [8]:
sds.add_objects(description='gpd_geometries',
dimension_list=['space'],
object_list=gdfRegions.geometry)
spr.add_region_centroids(sds, spatial_dim='space') 
spr.add_centroid_distances(sds)

In [9]:
aggregation_dict, precomputed_dist_matrix = spg.perform_parameter_based_grouping(sds, 
                                                        dimension_description='space',
                                                        linkage='complete')

 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:239: UserWarning:Variable gpd_geometries has dimensions ('space',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:239: UserWarning:Variable gpd_centroids has dimensions ('space',) which are not considered for spatial aggregation.
 c:\users\s.patil\documents\code\fine\FINE\spagat\grouping_utils.py:239: UserWarning:Variable centroid_distances has dimensions ('space', 'space_2') which are not considered for spatial aggregation.


The cophenetic correlation coefficient of the hiearchical clustering is  0.7121747361352841
Inconsistencies: [0.0, 0.0, 0.0, 0.7071067811865486, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865528, 0.0, 0.7071067811880829, 0.7071067811865469, 0.0, 0.7071067811865487, 0.7071067811865529, 0.7071067811865474, 0.7071067811865508, 0.7071067811865459, 0.6542387016444168, 0.0, 0.0, 0.7071067811865451, 0.707106781186556, 0.0, 0.0, 0.0, 0.0, 0.7747202761022581, 0.0, 0.7071067811865561, 0.8643695527546729, 0.7071067811865477, 0.7071067811862226, 0.0, 0.0, 0.7071067811865089, 0.7071067811865512, 0.707106781186535, 0.0, 0.0, 0.0, 0.7071067811865425, 0.7071067811865475, 1.0097331146617214, 0.707106781186548, 1.017155815024377, 0.9199254279365316, 0.7071067811874098, 0.0, 0.0, 0.0, 0.707106781186571, 1.0440295850902683, 0.7071067811865474, 0.7725255618182695, 0.7071067811865478, 1.1078974018426155, 0.7071067811829218, 0.7071067811865467, 1.046989499593277, 0.7071067811865565, 0.8525780709767388, 1.1280

In [10]:
precomputed_dist_matrix

array([[    0.        ,  1951.92922127,  4528.65204958, ...,
         6928.09056743,  4641.66107494,  6619.09352626],
       [ 1951.92922127,     0.        ,  3208.47017188, ...,
         7311.35756213,  4348.63930911,  6779.95469206],
       [ 4528.65204958,  3208.47017188,     0.        , ...,
         9569.04679329,  4162.40644317,  3179.79395763],
       ...,
       [ 6928.09056743,  7311.35756213,  9569.04679329, ...,
            0.        ,  7844.53812627, 13163.24598776],
       [ 4641.66107494,  4348.63930911,  4162.40644317, ...,
         7844.53812627,     0.        ,  4012.07451288],
       [ 6619.09352626,  6779.95469206,  3179.79395763, ...,
        13163.24598776,  4012.07451288,     0.        ]])

In [19]:
for i in range(len(sds.xr_dataset.space.values)):
    for j in range(i+1,len(sds.xr_dataset.space.values)):
        country_i, country_j = sds.xr_dataset.space.values[i][3:], sds.xr_dataset.space.values[j][3:]
        if (country_i in ['es', 'pt'] and country_j in ['es', 'pt']):
            print(f'{sds.xr_dataset.space.values[i]}, {sds.xr_dataset.space.values[j]}')
            print(precomputed_dist_matrix[i, j])
            print('======================================================')

01_es, 02_es
1951.9292212706978
01_es, 03_es
4528.652049583781
01_es, 04_es
3022.394509671156
01_es, 05_es
5284.551658261108
01_es, 06_es
4286.7354683142785
01_es, 07_es
5487.939090503262
01_es, 08_es
4892.618750956772
01_es, 09_es
4252.706041543552
01_es, 10_es
4096.642457321403
01_es, 11_es
4600.960261331383
01_es, 12_pt
2140.5982131979686
01_es, 13_pt
2987.5667115950673
02_es, 03_es
3208.4701718812566
02_es, 04_es
2014.666568998775
02_es, 05_es
3906.0099025430554
02_es, 06_es
3009.1666675333913
02_es, 07_es
3747.8972256484644
02_es, 08_es
3518.711467955378
02_es, 09_es
3309.3641629380445
02_es, 10_es
3479.670942646985
02_es, 11_es
2934.8142579792957
02_es, 12_pt
2264.397787572965
02_es, 13_pt
2816.080911779922
03_es, 04_es
2805.7954180421425
03_es, 05_es
808.1853822029063
03_es, 06_es
3125.4226364050146
03_es, 07_es
610.9302932771562
03_es, 08_es
875.1663988603796
03_es, 09_es
3328.692292112138
03_es, 10_es
3074.698007425213
03_es, 11_es
2996.780559307176
03_es, 12_pt
3365.316210258